In [ ]:

from pinecone import Pinecone, ServerlessSpec

from flask import Flask, render_template, request,jsonify
import PyPDF2
import re
import json
import numpy as np
import requests
from google.colab.output import eval_js
from sentence_transformers import SentenceTransformer
from google.colab import drive
from dotenv import load_dotenv
import os

load_dotenv()
# Initialize Pinecone client
# Access the environment variables
bearer_token_1 = os.getenv("BEARER_TOKEN")
bearer_token_2 = os.getenv("BEARER_TOKEN")
google_api = os.getenv("GOOGLE_API_KEY")
pinecone_api = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api)
index_name = "start1"
app = Flask(__name__)
# Create Pinecone index if it doesn't exist
def create_index_if_not_exists():
    try:
        # Check if index already exists
        indexes = pc.list_indexes()
        if index_name not in [index.name for index in indexes]:
            pc.create_index(
                name=index_name,
                dimension=1024,  # Dimensions for bge-large-en-v1.5
                metric="cosine",
                spec=ServerlessSpec(
                    cloud="aws",
                    region="us-east-1"
                )
            )
            print(f"Index '{index_name}' created successfully")
        else:
            print(f"Index '{index_name}' already exists")
    except Exception as e:
        print(f"Error creating index: {e}")
def get_drive_folder():
  drive.mount('/content/drive')
  folder = "/content/drive/MyDrive/data/Assesment_Dataset/data/data"

# Get a list of all items in the directory
  all_items = os.listdir(folder)

  # Filter the list to include only folders
  folders = [item for item in all_items if os.path.isdir(os.path.join(folder, item))]

  return folders
def get_folder_name(jd,folders):

    url = "https://api.langflow.astra.datastax.com/lf/ad6e8f21-6c0a-4de7-b59c-837f011e58f2/api/v1/run/e0666050-2c47-4082-bbb3-491a6eb51591?stream=false"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {bearer_token_1}"
    }

    data = {
        "input_value": "message",
        "output_type": "chat",
        "input_type": "chat",
        "tweaks": {
            "Agent-m8iEy": {
                "add_current_date_tool": True,
                "agent_description": "A smart AI assistant capable of analyzing job descriptions and matching relevant resumes.",
                "agent_llm": "Google Generative AI",
                "handle_parsing_errors": True,
                "input_value": '',
                "max_iterations": 15,
                "n_messages": 100,
                "order": "Ascending",
                "sender": "Machine and User",
                "sender_name": "",
                "session_id": "",
                "system_prompt": "You are an AI assistant designed to analyze job descriptions and match relevant resumes.\nEnsure your responses are concise and formatted in Markdown for readability.",
                "template": "**{sender_name}**: {text}",
                "verbose": True,
                "model_name": "learnlm-1.5-pro-experimental",
                "api_key": google_api,
                "temperature": 0.1,
                "tool_model_enabled": False
            },
            "ChatOutput-PeXVa": {
                "background_color": "",
                "chat_icon": "",
                "clean_data": True,
                "data_template": "{text}",
                "input_value": "",
                "sender": "Machine",
                "sender_name": "AI",
                "session_id": "",
                "should_store_message": True,
                "text_color": ""
            },
            "Prompt-aATiG": {
                "template": "    Based on the job description below, determine if resumes stored in a folder with the following name are relevant.\n    \n    Job Description:\n    {job_description}\n    \n    Folder Name:\n    {folder_name}\n    \n    Respond with only relevant \"Folder Name\" present in variable don't by you anything as output with no extra text.\n",
                "tool_placeholder": "",
                "job_description": jd,
                "folder_name": f"{folders}"
            }
        }
    }

    # response = requests.post(url, headers=headers, data=json.dumps(data))
    response = requests.post(url, headers=headers, data=json.dumps(data))

    response=response.json()
    chat_response = response["outputs"][0]["outputs"][0]["results"]["message"]["data"]["text"]
    # print(chat_response)
    # folder_names = chat_response.split('/n') # Assuming comma-separated values in chat_response
    # folder_names = [name.strip() for name in folder_names] # Remove leading/trail
    # print(folder_names)
    folder_names = chat_response.split('\n')  # Split by newline
    folder_names = [name.strip() for name in folder_names if name.strip()]
    return folder_names
# Function to extract text from PDF files in a folder (limited to max_files)
def extract_pdf_text(folder_name, max_files=5):
    """Extract text from PDF files in the specified folder, limited to max_files."""
    folder_path = f"/content/drive/MyDrive/data/Assesment_Dataset/data/data/{folder_name}"
    file_data = []

    # Check if the folder exists
    if not os.path.exists(folder_path):
        print(f"Folder {folder_path} does not exist.")
        return file_data

    # Get a list of all PDF files in the folder
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith(".pdf")]

    # Limit to max_files
    pdf_files = pdf_files[:max_files]

    # Process each file
    for filename in pdf_files:
        file_path = os.path.join(folder_path, filename)
        resume_text = ""

        # Open and process the PDF file
        try:
            with open(file_path, 'rb') as pdf_file:
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                # Get the number of pages
                num_pages = len(pdf_reader.pages)

                # Extract text from all pages
                for page in range(num_pages):
                    page_obj = pdf_reader.pages[page]
                    resume_text += page_obj.extract_text()

            file_data.append({
                "file_name": filename,
                "text": resume_text
            })
            print(f"Successfully extracted text from {filename}")
        except Exception as e:
            print(f"Error extracting text from {filename}: {e}")

    return file_data

# Function to extract text from TXT files in a folder (limited to max_files)
def extract_txt_text(folder_name, max_files=5):
    """Extract text from TXT files in the specified folder, limited to max_files."""
    folder_path = f"/content/drive/MyDrive/data/Assesment_Dataset/data/data/{folder_name}"
    file_data = []

    # Check if the folder exists
    if not os.path.exists(folder_path):
        print(f"Folder {folder_path} does not exist.")
        return file_data

    # Get a list of all TXT files in the folder
    txt_files = [f for f in os.listdir(folder_path) if f.endswith(".txt")]

    # Limit to max_files
    txt_files = txt_files[:max_files]

    # Process each file
    for filename in txt_files:
        file_path = os.path.join(folder_path, filename)
        resume_text = ""

        # Open and process the TXT file
        try:
            with open(file_path, 'r', encoding='utf-8') as txt_file:
                resume_text = txt_file.read()

            file_data.append({
                "file_name": filename,
                "text": resume_text
            })
            print(f"Successfully extracted text from {filename}")
        except Exception as e:
            print(f"Error extracting text from {filename}: {e}")

    return file_data

# Function to generate embeddings
def generate_embeddings(text):
    """Generate embeddings for the given text using the BGE model."""
    model = SentenceTransformer('BAAI/bge-large-en-v1.5')

    # Generate embeddings
    embedding = model.encode(text)
    return embedding

# Function to store embeddings in Pinecone
def store_embeddings(resume_data, folder_name):
    """Store embeddings in Pinecone with metadata."""
    try:
        # Get the index
        index = pc.Index(index_name)

        # Process each resume
        for resume in resume_data:
            file_name = resume["file_name"]
            text = resume["text"]

            # Generate embeddings
            embedding = generate_embeddings(text)

            # Create unique ID
            vector_id = f"{folder_name}_{file_name}_{hash(text) % 10000}"

            # Create metadata
            metadata = {
                "folder_name": folder_name,
                "file_name": file_name,
                "text_sample": text # Store a sample of the text
            }

            # Upsert the embedding with metadata
            index.upsert(
                vectors=[
                    {
                        "id": vector_id,
                        "values": embedding.tolist(),
                        "metadata": metadata
                    }
                ]
            )

            print(f"Successfully stored embedding for {file_name} in Pinecone")

        return True
    except Exception as e:
        print(f"Error storing embeddings: {e}")
        return False

# Function to process a specific folder
def process_folder(folder_name, max_files=5):
    """Process a specific folder containing resumes, limited to max_files per file type."""
    # Create index if it doesn't exist
    # create_index_if_not_exists()

    print(f"Processing folder: {folder_name}")

    # Process PDF files (up to max_files)
    pdf_data = extract_pdf_text(folder_name, max_files)
    if pdf_data:
        store_embeddings(pdf_data, folder_name)

    # Process TXT files (up to max_files)
    txt_data = extract_txt_text(folder_name, max_files)
    if txt_data:
        store_embeddings(txt_data, folder_name)

    print(f"Finished processing folder: {folder_name}")

# Function to rank resume
def rank_resume(result_details):
    """
    Rank a resume using the API

    Args:
        result_details (dict): Resume details to rank

    Returns:
        dict: Ranked resume with score
    """
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {bearer_token_2}',
    }

    params = {
        'stream': 'false',
    }

    json_data = {
        'input_value': 'message',
        'output_type': 'chat',
        'input_type': 'chat',
        'tweaks': {
            'Agent-ZGHPJ': {
                'add_current_date_tool': True,
                'agent_description': 'A helpful assistant with access to the following tools:',
                'agent_llm': 'Google Generative AI',
                'handle_parsing_errors': True,
                'input_value': '',
                'max_iterations': 15,
                'n_messages': 100,
                'order': 'Ascending',
                'sender': 'Machine and User',
                'sender_name': '',
                'session_id': '',
                'system_prompt': 'You are a helpful assistant that can use tools to answer questions and perform tasks.',
                'template': '{sender_name}: {text}',
                'verbose': True,
                # 'max_output_tokens': None,
                'model_name': 'learnlm-1.5-pro-experimental',
                'api_key': google_api,
                # 'top_p': None,
                'temperature': 0,
                # 'n': None,
                # 'top_k': None,
                'tool_model_enabled': False,
            },
            'Prompt-cwlsj': {
                'template': '\nExtract the following with only fields from the given input also refine score as per jd matching if you want and return valid JSON:\n\n- id\n- score\n- folder\n- file\n- preview\n- name\n- skills (as a list)\n- experience\n- education\n- location\n\nSet any missing field to null. Only output JSON.\n\nInput:\n\\"\\"\\"\n{result_details}\n\\"\\"\\"\n\n',
                'tool_placeholder': '',
                'result_details': json.dumps(result_details),
            },
            'ChatOutput-NMGIM': {
                'background_color': '',
                'chat_icon': '',
                'clean_data': True,
                'data_template': '{text}',
                'sender': 'Machine',
                'sender_name': 'AI',
                'session_id': '',
                'should_store_message': True,
                'text_color': '',
            },
        },
    }

    try:
        response = requests.post(
            'https://api.langflow.astra.datastax.com/lf/ad6e8f21-6c0a-4de7-b59c-837f011e58f2/api/v1/run/017d81dc-f6ba-4eeb-b27f-f363fb7a6bba',
            params=params,
            headers=headers,
            json=json_data,
        )

        response_data = response.json()
        chat_response = response_data["outputs"][0]["outputs"][0]["results"]["message"]["data"]["text"]
        match = re.search(r'{.*}', chat_response, re.DOTALL)
        if match:
            result_dict = json.loads(match.group(0))
            return result_dict
        else:
            return {"id": result_details["id"], "score": result_details["score"]}
    except Exception as e:
        print(f"Error ranking resume: {e}")
        return {"id": result_details["id"], "score": result_details["score"]}

# Function to search resumes across multiple folders
def search_resumes(folder_names, query_text, top_k=5):
    """
    Search for resumes in multiple folders and return ranked results

    Args:
        folder_names (list): List of folder names containing resumes
        query_text (str): Query text to search for
        top_k (int): Number of top results to return

    Returns:
        list: Ranked resumes with their details
    """
    all_results = []

    # First, process all folders to ensure they're indexed
    for folder in folder_names:
        process_folder(folder, max_files=5)

    # Generate embeddings for the query
    query_embedding = generate_embeddings(query_text)

    try:
        # Get the index
        index = pc.Index(index_name)

        # Search for similar vectors
        results = index.query(
            vector=query_embedding.tolist(),
            top_k=top_k * len(folder_names),
            include_metadata=True
        )

        # Process each result
        if results and 'matches' in results:
            for match in results['matches']:
                result_details = {
                    "id": match['id'],
                    "score": float(match['score']),  # Convert to float to ensure it's JSON serializable
                    "folder": match['metadata'].get('folder_name'),
                    "file": match['metadata'].get('file_name'),
                    "preview": match['metadata'].get('text_sample')[:500]  # Limit text sample size
                }

                # Pass each result to rank_resume function
                ranked_result = rank_resume(result_details)

                # Add to all results
                all_results.append(ranked_result)

        # Sort results by score in descending order
        sorted_results = sorted(all_results, key=lambda x: float(x.get('score', 0)), reverse=True)

        # Return top results
        return sorted_results[:top_k]

    except Exception as e:
        print(f"Error searching for resumes: {e}")
        return []

@app.route("/", methods=["GET", "POST"])
def index():
    results = []
    if request.method == "POST":
        jd = request.form.get("jd")
        create_index_if_not_exists()
        folders = get_drive_folder()
        folder_names = get_folder_name(jd, folders)
        results = search_resumes(folder_names, jd, top_k=3)
    return render_template("index.html", results=results)

if __name__ == '__main__':
    port = 9800  # Choose a port
    print(f"Flask App Running: {eval_js('google.colab.kernel.proxyPort({})'.format(port))}")
    app.run(port=port, debug=False)
# # Main function
# def main():
#     # List of folders to process
#     # folder_names = ["ENGINEERING"]  # Replace with your actual folder names

#     # Search query
#     jd= input('Enter job description')
#     create_index_if_not_exists()
#     folders=get_drive_folder()
#     folder_names=get_folder_name(jd,folders)
#     print(folder_names)
#     print(f"Searching for: {jd}")

#     # Search for resumes across all folders
#     results = search_resumes(folder_names, jd, top_k=3)

#     # Display results
#     if results:
#         print("\nTop ranked resumes:")
#         for i, result in enumerate(results, 1):
#             print(f"\nRank {i}:")
#             print(f"I: {result.get('id')}")
#             print(f"Score: {result.get('score')}")
#             print(f"Name: {result.get('name')}")
#             print(f"Folder: {result.get('folder')}")
#             print(f"File: {result.get('file')}")
#             print(f"Skills: {result.get('skills')}")
#             print(f"Experience: {result.get('experience')}")
#             print(f"Education: {result.get('education')}")
#             print(f"Location: {result.get('location')}")
#     else:
#         print("No results found.")

# if __name__ == "__main__":
#     main()


